In [4]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\dhyan\Desktop\SJU\COMPETETIONS\BANGLORE TRAFFIC BYTE


In [5]:

# !pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.17  Python-3.9.7 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
Setup complete  (16 CPUs, 15.2 GB RAM, 427.8/474.7 GB disk)


In [6]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [7]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [5]:
# !pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


In [6]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [7]:
from typing import List
import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [8]:
os.chdir(HOME)

In [9]:
MODEL = r".\best.pt"

In [10]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

Model summary (fused): 218 layers, 25,843,813 parameters, 0 gradients, 78.7 GFLOPs


In [11]:
SOURCE_VIDEO_PATH = "4.mp4"
TARGET_VIDEO_PATH="output/4.mp4"

In [12]:
from supervision.video.dataclasses import VideoInfo

VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1920, height=1080, fps=25, total_frames=500)

In [14]:
# from tqdm.notebook import tqdm

# class_names = ["Bicycle", "Bus", "Cars", "LCV", "Three-Wheeler", "Truck", "Two-Wheeler"]
# byte_tracker = BYTETracker(BYTETrackerArgs())
# generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
# video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)


# with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
#     for frame in tqdm(generator, total = video_info.total_frames):

#         results = model(frame)

#         detections = Detections(
#             xyxy=results[0].boxes.xyxy.cpu().numpy(),
#             confidence=results[0].boxes.conf.cpu().numpy(),
#             class_id=results[0].boxes.cls.cpu().numpy().astype(int)
#         )

#         tracks = byte_tracker.update(
#             output_results=detections2boxes(detections=detections),
#             img_info=frame.shape,
#             img_size=frame.shape
#         )
#         tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
#         print(tracker_id)
#         detections.tracker_id = np.array(tracker_id)

#         if len(detections.xyxy) > 0 and len(tracker_id) > 0:
#             labels = [
#                 f"#{tracker_id} {class_names[class_id]} {confidence:0.2f}"
#                 for _, confidence, class_id, tracker_id
#                 in detections
#             ]
    
#             frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
#             sink.write_frame(frame)


In [15]:
LINE_START = Point(300, 300)
LINE_END = Point(300+1300, 700)

In [18]:
from tqdm.notebook import tqdm
from line_c import LineCounter,LineCounterAnnotator

class_names = ["Bicycle", "Bus", "Cars", "LCV", "Three-Wheeler", "Truck", "Two-Wheeler"]
CLASS_ID = [0,1,2,3,4,5,6]
# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create LineCounter instance
line_counter = LineCounter(start=LINE_START, end=LINE_END, class_names=class_names)
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)

# open target video file
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        # model prediction on single frame and conversion to supervision Detections
        results = model(frame)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # filtering out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # format custom labels
        labels = [
            f"#{tracker_id} {class_names[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        # updating line counter
        line_counter.update(detections=detections)
        vehicles_in = line_counter.in_count
        vehicles_out = line_counter.out_count
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        line_annotator.annotate(frame=frame, line_counter=line_counter, class_names=class_names)
        sink.write_frame(frame)
    print(f"In Count = {vehicles_in}")
    print(f"Out Count = {vehicles_out}")

  0%|          | 0/500 [00:00<?, ?it/s]


0: 384x640 2 Two-Wheelers, 80.0ms
Speed: 4.0ms preprocess, 80.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Two-Wheeler, 70.4ms
Speed: 2.8ms preprocess, 70.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Two-Wheeler, 73.6ms
Speed: 2.0ms preprocess, 73.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Two-Wheeler, 46.1ms
Speed: 2.0ms preprocess, 46.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cars, 1 Two-Wheeler, 41.2ms
Speed: 1.0ms preprocess, 41.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Three-Wheeler, 1 Two-Wheeler, 41.2ms
Speed: 2.0ms preprocess, 41.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 33.8ms
Speed: 1.0ms preprocess, 33.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.0ms
Speed: 1.5ms prepro